In [1]:
#make sure to pip install all the required stuff!!!!
from yahoo_fin.stock_info import get_live_price
from yahoo_fin import options as op
import pandas as pd
from datetime import datetime, timedelta
import math
import random
import requests

In [2]:
#get contract and underlying information 
#this works best with stock option contracts!!
#AAPL240719C00105000 <- example contract that you can use as input
contractInfo = input("Enter the contract symbol for your option: ")
numContracts = int(input('Enter the number of contracts: '))

#global things that will be used later
tickerSymbol = contractInfo[0:4]
underlyingPrice = get_live_price(tickerSymbol)
year = int(contractInfo[4:6])
month = int(contractInfo[6:8])
day = int(contractInfo[8:10])
expDate = datetime(2000 + year, month, day)
callOrPut = contractInfo[10]
strike = int(contractInfo[11:]) / 1000 
todaysDate = datetime.today()

#get IV for option
def getIV():
  #get option chain
  chain = []
  if callOrPut == 'C': 
    chain = op.get_calls(tickerSymbol)
  else: 
    chain = op.get_puts(tickerSymbol)
  
  #isolate volatility column for the option with the matching strike price
  chain = pd.DataFrame(chain)
  mask = chain['Strike'] == strike
  volPercent = chain[mask].head().to_numpy()[0][10]
  return volPercent[0:len(volPercent) - 1]

In [3]:
#approximate the interest rate for X months offset
def getInterestRate(month):
  #rates are based on the yield curve for march 11
  #https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=2024
  yieldRateMarch11 = {
    1: 5.5,
    2: 5.47,
    3: 5.48,
    4: 5.41,
    6: 5.35,
    12: 4.95,
    36: 4.51,
    60: 4.26,
    84: 4.08, #7 year (what r u doing if ur buying 7 year options)
  }
  dates = [1, 2, 3, 4, 6, 12, 36, 60, 84]
  
  #get closest month to approximate the interest rate
  minDiff = 100000000
  rate = 5.5 #equals month 1 by default
  for date in reversed(dates):
    if abs(date - month) < minDiff:
      minDiff = abs(date - month)
      rate = yieldRateMarch11[date]
  
  return rate #defaut is returning the first month
  
#black scholes formula to estimate delta of option
#lowkey this formula doesn't really do a good job, and in reality you can just get delta from an api
#we calculated it by hand for some reason using the following formula:
#https://www.simtrade.fr/blog_simtrade/option-greeks-delta/
def getBlackScholesDelta():
  S = float(underlyingPrice) #underlying price
  K = float(strike) #strike
  r = getInterestRate((expDate - todaysDate).days / 30) #risk free return (assume 30 days in a month)
  T = float((expDate - todaysDate).days) #days to maturity
  vol = float(getIV()) / 100 #underlying vol
  
  delta = (math.log(S/K) + T * (r + math.pow(vol, 2) / 2)) / (vol * math.sqrt(T))
  
  #delta is negative for puts
  if callOrPut == 'P':
    return 1 - delta
  return delta

In [4]:
history = [] #all purchases (adjustments) to the initial options spread

#adjust by the appropriate number of contracts based on the delta
def adjust(delta):
  if delta > 0: #postive delta is balanced by selling underlying 
    print(f"Sell {delta} shares of underlying")
    buySellUnderlying(-1 * delta)
  elif delta < 0: #negative delta is balanced by buying underlying
    print(f"Buy {delta * -1} shares of underlying")
    buySellUnderlying(-1 * delta)
  else:
    print("No adjustments are required")

#this method is where you actually buy the stuff and make an emergency trade
def buySellUnderlying(amount):
  history.append([amount, get_live_price(tickerSymbol)])
  #here is where you would send a notification or email to someone to trade!

#calculate the delta of the entire position
def getPositionDelta():
  #get delta of shares
  numShares = 0
  for purchase in history:
    numShares += purchase[0]
  
  #add delta of shares to delta of options
  return getBlackScholesDelta() * numContracts + numShares

In [5]:
#initial spread
optionDelta = getBlackScholesDelta()
print(f"Your initial spread is: ")
adjust(optionDelta) #when you purchase options, you also want to buy/sell underlying so you can start delta neutral

Your initial spread is: 
Sell 40.70321136114257 shares of underlying


In [6]:
#the following code simulates the tool
#in reality, it would be integrated into the hokie terminal
#and prices/delta would change based on the market

initialDaysTillExpiration = (expDate - todaysDate).days

#when delta exceeds this boundary, you adjust
threshold = float(input("Enter the threshold for the delta as decimal between 0.1 and 1(determines how frequently you will adjust): "))

#assuming one adjustment per day
for i in range(initialDaysTillExpiration - 1):   
  todaysDate += timedelta(days=1) #new day!
  underlyingPrice = underlyingPrice + random.randint(-10, 10) #simulate variation in prices
  
  #check if delta has deviated from 0 by the threshold
  positionDelta = getPositionDelta()
  print("----------------------------------------------------------")
  print(f"Today is {todaysDate.strftime('%m/%d/%Y')} and the delta is {positionDelta}!")
  
  #adjust if required
  if abs(positionDelta) > threshold:
    adjust(positionDelta)

----------------------------------------------------------
Today is 03/12/2024 and the delta is 162.02327702326173!
Sell 162.02327702326173 shares of underlying
----------------------------------------------------------
Today is 03/13/2024 and the delta is -0.7902956959070195!
Buy 0.7902956959070195 shares of underlying
----------------------------------------------------------
Today is 03/14/2024 and the delta is -0.7934276782254699!
Buy 0.7934276782254699 shares of underlying
----------------------------------------------------------
Today is 03/15/2024 and the delta is -0.8036518916824775!
Buy 0.8036518916824775 shares of underlying
----------------------------------------------------------
Today is 03/16/2024 and the delta is -0.8009415936718369!
Buy 0.8009415936718369 shares of underlying
----------------------------------------------------------
Today is 03/17/2024 and the delta is -0.7971437026672277!
Buy 0.7971437026672277 shares of underlying


KeyboardInterrupt: 